In [1]:
'''
This Python code loads localization data of particles from three different channels of an experiment, 
performs particle tracking, and creates a plot of the tracked particles.
We use a sparse sample and gather many localizations by raster-tiling the sample
Beads are localized per frame, and localizations are then combined and tracked.

'''




'\nThis Python code loads localization data of particles from three different channels of an experiment, \nperforms particle tracking, and creates a plot of the tracked particles.\nWe use a sparse sample and gather many localizations by raster-tiling the sample\nBeads are localized per frame, and localizations are then combined and tracked.\n\n'

In [2]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yaml as yaml
import trackpy as tp
# import naclib.stpol
# import naclib.util

### Define functions for loading data and tracking

def load_locs(file, trackID='track.id', t='t'):
    """
    Load (simulated) loc files into dataframes and change dataframe.columns
    from swift-compatible to trackpy compatible and for better codability.
    """
    locs = pd.read_csv(file)
    info = load_info(file)
    if trackID in locs.columns:
        locs.rename(columns={trackID: 'particle'}, inplace=True)
        locs.rename(columns={t: 'frame'}, inplace=True)
    return (locs, info)

def load_info(path, qt_parent=None):
    path_base, path_extension = os.path.splitext(path)
    filename = path_base + ".yaml"
    with open(filename, "r") as info_file:
        info = list(yaml.load_all(info_file, Loader=yaml.FullLoader))
    return info

# Define a function to perform particle tracking on localization data
def apply_tracking(df_locs_all):
    df_linked = link.link_locs_trackpy(df_locs_all, search=20, memory=0)
    return df_linked


# Define the path to the directory containing the experiment data
path = r'D:\20230421 tetraspeck\Run00017'

# Generate a list of file paths for the localization CSV of the files of all three channels
paths = glob.glob(path+'/*locs.csv')

# Load localization data from the CSV files
df_locs0, info = load_locs(paths[0])
df_locs1, info = load_locs(paths[1])
df_locs2, info = load_locs(paths[2])

# Combine the localizations from all channels into a single DataFrame and rename columns
# We want to assign the bead position in one channel to the respective positions in the other channels
# Therefore, we use the 'frame' column for the different channels
# The actual frame column is renamed to 'grid' and is only   

df_locs_all = pd.concat([df_locs0, df_locs1, df_locs2], keys=['0', '1', '2']).reset_index(
    level=0).rename(columns={'t': 'grid', 'level_0': 'frame'})

df_locs_all.head()

,frame,grid,x,y,intensity,sx,sy,bg,lpx,lpy,ellipticity,net_gradient,loc_precision,nearest_neighbor
0,0,0,51.086500,428.82623,19691.840,1.781357,1.490991,78.633700,0.026384,0.021695,0.163003,30926.648,0.024039,57.037067
1,0,0,73.333150,336.57140,28729.945,1.770485,1.656289,90.415726,0.021337,0.019845,0.064500,36668.336,0.020591,77.062232
2,0,0,85.709526,474.15253,11865.251,1.561334,1.463506,49.831050,0.029471,0.027475,0.062657,22742.012,0.028473,43.237782
3,0,0,87.909800,517.33430,20391.260,1.654282,1.545324,51.825930,0.023271,0.021661,0.065864,35771.070,0.022466,43.237782
4,0,0,111.674840,661.14050,12584.068,1.700517,1.629641,29.190310,0.030374,0.029042,0.041679,19392.172,0.029708,103.659531


In [3]:


# Apply the tracking function to each group of localizations by grid value
df_tracks = df_locs_all.groupby('grid').apply(
    apply_tracking).reset_index(level=0, drop=True)

# Filter the tracked localizations to only include tracks with more than 2 localizations
df_tracks = df_tracks.loc[df_tracks.loc_count > 2]

# Ensure each track has a unique ID
df_tracks = tools.get_unique_trackIDs(df_tracks, group='grid')

# Add a column for segment ID identical to the track ID
df_tracks['seg.id'] = df_tracks['track.id']

# Save the tracked localizations to a CSV file
df_tracks.to_csv(r'D:\20230421 tetraspeck\Run00017\tracked.csv')

# Create a scatter plot of the tracked localizations
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(111)
ax = df_tracks.plot.scatter(x='x', y='y', c='t', colormap='brg',
                            marker='.', colorbar=False, ax=ax, alpha=0.3)
ax.set_xlabel('x (px)')
ax.set_ylabel('y (px)')
ax.set_aspect('equal')

# Save the plot as a PNG file
plt.savefig(os.path.join(path, 'plots', runName+'_uncorrected.png'),
            dpi=200, bbox_inches='tight', transparent=False)

# Create a list of x and y coordinates of localizations at time points 0, 1, and 2
points = [df_tracks.loc[df_tracks.t == i, ['x', 'y']].values for i in [0, 1, 2]]

NameError: name 'link' is not defined